# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

02-01-2023


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2023-02-02 04:20:54,33.93911,67.709953,208552,7882,NaN,NaN,Afghanistan,535.733079,3.779393
1,NaN,NaN,NaN,Albania,2023-02-02 04:20:54,41.15330,20.168300,334177,3596,NaN,NaN,Albania,11612.238516,1.076076
2,NaN,NaN,NaN,Algeria,2023-02-02 04:20:54,28.03390,1.659600,271385,6881,NaN,NaN,Algeria,618.879236,2.535512
3,NaN,NaN,NaN,Andorra,2023-02-02 04:20:54,42.50630,1.521800,47839,165,NaN,NaN,Andorra,61915.485666,0.344907
4,NaN,NaN,NaN,Angola,2023-02-02 04:20:54,-11.20270,17.873900,105184,1931,NaN,NaN,Angola,320.036336,1.835831


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/23/23,1/24/23,1/25/23,1/26/23,1/27/23,1/28/23,1/29/23,1/30/23,1/31/23,2/1/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,208097,208289,208324,208324,208432,208435,208435,208502,208545,208552
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334101,334113,334124,334135,334144,334153,334157,334157,334167,334177
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271346,271354,271360,271364,271369,271369,271376,271376,271378,271385


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/23/23,1/24/23,1/25/23,1/26/23,1/27/23,1/28/23,1/29/23,1/30/23,1/31/23,2/1/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7870,7871,7872,7872,7875,7876,7876,7879,7882,7882
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/23/23,1/24/23,1/25/23,1/26/23,1/27/23,1/28/23,1/29/23,1/30/23,1/31/23,2/1/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/23/23,1/24/23,1/25/23,1/26/23,1/27/23,1/28/23,1/29/23,1/30/23,1/31/23,2/1/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,19389,19389,19471,19471,19471,19471,19471,19471,19471,19530
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,68764,68764,68983,68983,68983,68983,68983,68983,68983,69187
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7258,7258,7299,7299,7299,7299,7299,7299,7299,7339


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/23/23,1/24/23,1/25/23,1/26/23,1/27/23,1/28/23,1/29/23,1/30/23,1/31/23,2/1/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,230,230,230,230,230,230,230,230,230,230
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,722,722,723,723,723,723,723,723,723,723
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,103,103,103,103,103,103,103,103,103,103


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
678,1001.0,Autauga,Alabama,US,2023-02-02 04:20:54,32.539527,-86.644082,19530,230,NaN,NaN,"Autauga, Alabama, US",34956.773882,1.177675
715,1075.0,Lamar,Alabama,US,2023-02-02 04:20:54,33.779950,-88.096680,4675,73,NaN,NaN,"Lamar, Alabama, US",33864.541833,1.561497
716,1077.0,Lauderdale,Alabama,US,2023-02-02 04:20:54,34.901719,-87.656247,30057,419,NaN,NaN,"Lauderdale, Alabama, US",32413.807978,1.394018


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,102451346,1109591,0.0
India,44684248,530741,0.0
France,39733692,165327,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,102451346,1109591,0.0,101341755.0,2023-02-02 04:20:54,37.934269,-91.444083
India,44684248,530741,0.0,44153507.0,2023-02-02 04:20:54,23.088275,81.806127
France,39733692,165327,0.0,39568365.0,2023-02-02 04:20:54,6.430808,-34.730285
Germany,37796790,165865,0.0,37630925.0,2023-02-02 04:20:54,51.571844,10.277881
Brazil,36837943,697200,0.0,36140743.0,2023-02-02 04:20:54,-12.669522,-48.480493


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,11987678,100109,0.0
Texas,8372997,92711,0.0
Florida,7443954,84927,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,11987678,100109,0.0,11887569.0,2023-02-02 04:20:54,37.843962,-120.728594
Texas,8372997,92711,0.0,8280286.0,2023-02-02 04:20:54,31.660643,-98.653069
Florida,7443954,84927,0.0,7359027.0,2023-02-02 04:20:54,28.940755,-82.700744
New York,6716939,76477,0.0,6640462.0,2023-02-02 04:20:54,42.544151,-75.474183
Illinois,4019768,41088,0.0,3978680.0,2023-02-02 04:20:54,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3677849,35294,0.0
Florida,Miami-Dade,1524998,12049,0.0
Illinois,Cook,1507271,15161,0.0
Arizona,Maricopa,1497504,18726,0.0
Texas,Harris,1262246,11547,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3677849,35294,0.0,3642555.0,2023-02-02 04:20:54,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1524998,12049,0.0,1512949.0,2023-02-02 04:20:54,25.611236,-80.551706,12086.0
Illinois,Cook,1507271,15161,0.0,1492110.0,2023-02-02 04:20:54,41.841448,-87.816588,17031.0
Arizona,Maricopa,1497504,18726,0.0,1478778.0,2023-02-02 04:20:54,33.348359,-112.491815,4013.0
Texas,Harris,1262246,11547,0.0,1250699.0,2023-02-02 04:20:54,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-30,208502,334157,271376,47839,105184,11,9106,10037135,446008,11295446,...,1032710,250563,12014,551542,11526461,703228,535,11945,340582,261606
2023-01-31,208545,334167,271378,47839,105184,11,9106,10037135,446008,11295446,...,1032710,250567,12014,551587,11526497,703228,535,11945,340763,261606
2023-02-01,208552,334177,271385,47839,105184,11,9106,10037135,446008,11295446,...,1032710,250586,12014,551587,11526508,703228,535,11945,340944,262324


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-30,7879,3596,6881,165,1931,0,146,130421,8717,18615,...,7609,1637,14,5845,43186,5708,0,2159,4046,5652
2023-01-31,7882,3596,6881,165,1931,0,146,130421,8717,18615,...,7609,1637,14,5845,43186,5708,0,2159,4047,5652
2023-02-01,7882,3596,6881,165,1931,0,146,130421,8717,18615,...,7609,1637,14,5845,43186,5708,0,2159,4048,5658


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/23/23,1/24/23,1/25/23,1/26/23,1/27/23,1/28/23,1/29/23,1/30/23,1/31/23,2/1/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,19389,19389,19471,19471,19471,19471,19471,19471,19471,19530
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4635,4635,4661,4661,4661,4661,4661,4661,4661,4675
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,29801,29801,29954,29954,29954,29954,29954,29954,29954,30057
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,9243,9243,9289,9289,9289,9289,9289,9289,9289,9323
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,46420,46420,46651,46651,46651,46651,46651,46651,46651,46938


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-30,1610535,303575,8320,2398200,996401,11983168,1748568,966353,325872,49,...,2477159,8336211,1081346,150190,24512,2251042,1904198,634285,1982587,183774
2023-01-31,1610535,304368,8320,2398200,996758,11986095,1749006,966860,325872,49,...,2477159,8336979,1081346,150190,24540,2260796,1904198,634541,1983463,183173
2023-02-01,1617850,304368,8320,2401287,996887,11987678,1749557,967429,326847,49,...,2477159,8372997,1081346,150668,24558,2260796,1904198,634853,1984248,183173


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2023-01-30       19471   68983    7299  7919  18255    2970   6410   40586   
2023-01-31       19471   68983    7299  7919  18255    2970   6410   40586   
2023-02-01       19530   69187    7339  7967  18349    2991   6449   40813   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2023-01-30        10656     6668  ...    7648   2255     9789     2305   
2023-01-31        10656     6668  ...    7627   2237     9784     2298   
2023-02-01        10705     6688  ...    7627   2237     9784     2298   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2023-01-30          12454  12082  6353          0     2737   1885  
2023-01-31          12351  12058  6317          0     2729   1880  
2023-02-01          12351  12058  6317          0     2729   1880  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-30,20870,1473,34,32775,12842,100027,14032,11941,3254,0,...,28977,92322,5253,889,129,23088,15380,7836,16097,1977
2023-01-31,20870,1473,34,32775,12852,100078,14034,11964,3254,0,...,28977,92322,5253,889,129,23135,15380,7849,16116,1979
2023-02-01,20882,1473,34,32882,12865,100109,14046,11964,3265,0,...,28977,92711,5253,894,129,23135,15380,7852,16124,1979


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2023-01-30         230     723     103  109    261      54    131     674   
2023-01-31         230     723     103  109    261      54    131     674   
2023-02-01         230     723     103  109    261      54    131     674   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2023-01-30          170       89  ...     152     44       87       28   
2023-01-31          170       89  ...     152     44       88       28   
2023-02-01          170       89  ...     152     44       88       28   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2023-01-30            137    16    43          0       47     22  
2023-01-31            137    16    43          0       47     22  
2023-02-01            137    16    43          0       47     22  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-30,0.000321,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000441,0.0,0.0,...,0.001094,0.000220,0.0,0.000236,0.000005,0.0,0.0,0.0,0.001205,0.000000
2023-01-31,0.000206,0.00003,0.000007,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000016,0.0,0.000082,0.000003,0.0,0.0,0.0,0.000531,0.000000
2023-02-01,0.000034,0.00003,0.000026,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000076,0.0,0.000000,0.000001,0.0,0.0,0.0,0.000531,0.002745


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-30,0.000381,0.0,0.0,0.0,0.0,NaN,0.0,0.000207,0.0,0.0,...,0.001053,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.000495,0.000000
2023-01-31,0.000381,0.0,0.0,0.0,0.0,NaN,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.000247,0.000000
2023-02-01,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.000247,0.001062


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-30,0.000000,0.000000,0.0,0.000000,0.000135,0.000000,0.000538,0.000148,0.000000,0.0,...,0.0,0.000095,0.0,0.000000,0.001512,0.000000,0.0,0.001024,0.000648,0.00000
2023-01-31,0.000000,0.002612,0.0,0.000000,0.000358,0.000244,0.000251,0.000525,0.000000,0.0,...,0.0,0.000092,0.0,0.000000,0.001142,0.004333,0.0,0.000404,0.000442,-0.00327
2023-02-01,0.004542,0.000000,0.0,0.001287,0.000129,0.000132,0.000315,0.000589,0.002992,0.0,...,0.0,0.004320,0.0,0.003183,0.000733,0.000000,0.0,0.000492,0.000396,0.00000


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State  Alabama                                                   \
Admin2          Autauga   Baldwin  Barbour      Bibb    Blount   Bullock   
2023-01-30      0.00000  0.000000  0.00000  0.000000  0.000000  0.000000   
2023-01-31      0.00000  0.000000  0.00000  0.000000  0.000000  0.000000   
2023-02-01      0.00303  0.002957  0.00548  0.006061  0.005149  0.007071   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2023-01-30      0.000000  0.000000  0.000000  0.000000  ...  0.000000   
2023-01-31      0.000000  0.000000  0.000000  0.000000  ... -0.002746   
2023-02-01      0.006084  0.005593  0.004598  0.002999  ...  0.000000   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2023-01-30      0.000000  0.000000  0.000000    0.00000  0.000000  0.000000   
2023-01-31     -0.007982 -0.000511 -0.003037   -0.00827 -0.001986 -0.005667   
2023-02-01      0.000000  0.000000  0.000000    0.00000  0.000000  0.000000   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2023-01-30            NaN  0.000000  0.000000  
2023-01-31            NaN -0.002923 -0.002652  
2023-02-01            NaN  0.000000  0.000000  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-30,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000571,0.000000,0.00000,NaN,...,0.0,0.000076,0.0,0.000000,0.0,0.000000,0.0,0.001790,0.000000,0.000000
2023-01-31,0.000000,0.0,0.0,0.000000,0.000779,0.00051,0.000142,0.001926,0.00000,NaN,...,0.0,0.000000,0.0,0.000000,0.0,0.002036,0.0,0.001659,0.001180,0.001012
2023-02-01,0.000575,0.0,0.0,0.003265,0.001012,0.00031,0.000855,0.000000,0.00338,NaN,...,0.0,0.004214,0.0,0.005624,0.0,0.000000,0.0,0.000382,0.000496,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2023-01-30         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2023-01-31         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2023-02-01         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                            \
Admin2         Chambers Cherokee  ...    Park Platte  Sheridan Sublette   
2023-01-30          0.0      0.0  ...     0.0    0.0  0.000000      0.0   
2023-01-31          0.0      0.0  ...     0.0    0.0  0.011494      0.0   
2023-02-01          0.0      0.0  ...     0.0    0.0  0.000000      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2023-01-30            0.0   0.0   0.0        NaN      0.0    0.0  
2023-01-31            0.0   0.0   0.0        NaN      0.0    0.0  
2023-02-01            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-30,0.000205,0.000010,0.000009,0.000050,0.000007,0.0,0.0,0.000224,0.0,0.000067,...,0.000555,0.000135,0.0,0.000129,0.000003,0.0,0.0,0.0,0.000687,0.000049
2023-01-31,0.000206,0.000020,0.000008,0.000025,0.000003,0.0,0.0,0.000112,0.0,0.000034,...,0.000278,0.000076,0.0,0.000105,0.000003,0.0,0.0,0.0,0.000609,0.000024
2023-02-01,0.000120,0.000025,0.000017,0.000013,0.000002,0.0,0.0,0.000056,0.0,0.000017,...,0.000139,0.000076,0.0,0.000053,0.000002,0.0,0.0,0.0,0.000570,0.001385


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-30,0.000236,0.0,0.0,0.0,0.000004,NaN,0.0,0.000105,0.0,0.000941,...,0.000528,0.0,0.0,0.000038,0.0,0.0,NaN,0.0,0.000282,0.000021
2023-01-31,0.000309,0.0,0.0,0.0,0.000002,NaN,0.0,0.000053,0.0,0.000470,...,0.000264,0.0,0.0,0.000019,0.0,0.0,NaN,0.0,0.000265,0.000010
2023-02-01,0.000154,0.0,0.0,0.0,0.000001,NaN,0.0,0.000026,0.0,0.000235,...,0.000132,0.0,0.0,0.000009,0.0,0.0,NaN,0.0,0.000256,0.000536


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-30,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2023-01-31,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2023-02-01,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-30,0.000075,0.000017,0.000046,0.000023,0.000182,0.000049,0.000327,0.000272,0.000039,0.0,...,0.000106,0.000099,0.000037,0.000053,0.000889,0.000037,0.00004,0.000578,0.000369,0.000008
2023-01-31,0.000037,0.001315,0.000023,0.000012,0.000270,0.000147,0.000289,0.000398,0.000019,0.0,...,0.000053,0.000095,0.000019,0.000026,0.001015,0.002185,0.00002,0.000491,0.000405,-0.001631
2023-02-01,0.002290,0.000657,0.000011,0.000649,0.000200,0.000139,0.000302,0.000493,0.001506,0.0,...,0.000026,0.002208,0.000009,0.001605,0.000874,0.001093,0.00001,0.000491,0.000400,-0.000816


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2023-01-30      0.000067  0.000050  0.000089  0.000060  0.000108  0.000075   
2023-01-31      0.000033  0.000025  0.000045  0.000030  0.000054  0.000037   
2023-02-01      0.001532  0.001491  0.002763  0.003046  0.002602  0.003554   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2023-01-30      0.000156  0.000081  0.000085  0.000095  ...  0.000005   
2023-01-31      0.000078  0.000041  0.000042  0.000048  ... -0.001370   
2023-02-01      0.003081  0.002817  0.002320  0.001523  ... -0.000685   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2023-01-30      0.000003  0.000012  0.000007   0.000008  0.000011  0.000009   
2023-01-31     -0.003989 -0.000249 -0.001515  -0.004131 -0.000988 -0.002829   
2023-02-01     -0.001995 -0.000125 -0.000758  -0.002066 -0.000494 -0.001414   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2023-01-30      -0.503937  0.000012  0.000004  
2023-01-31      -0.503937 -0.001456 -0.001324  
2023-02-01      -0.503937 -0.000728 -0.000662  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-30,0.000018,0.000097,0.0,0.000069,0.000186,0.000086,0.000321,0.000022,0.000055,NaN,...,0.000055,0.000073,0.000114,0.000089,5.000000e-07,0.000043,0.000068,0.000955,0.000099,0.000028
2023-01-31,0.000009,0.000048,0.0,0.000035,0.000483,0.000298,0.000232,0.000974,0.000027,NaN,...,0.000028,0.000036,0.000057,0.000044,3.000000e-07,0.001040,0.000034,0.001307,0.000640,0.000520
2023-02-01,0.000292,0.000024,0.0,0.001650,0.000747,0.000304,0.000543,0.000487,0.001704,NaN,...,0.000014,0.002125,0.000029,0.002834,1.000000e-07,0.000520,0.000017,0.000845,0.000568,0.000260


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State Alabama                                                 \
Admin2         Autauga   Baldwin Barbour          Bibb Blount Bullock   
2023-01-30         0.0  0.000022     0.0  1.100000e-06    0.0     0.0   
2023-01-31         0.0  0.000011     0.0  6.000000e-07    0.0     0.0   
2023-02-01         0.0  0.000005     0.0  3.000000e-07    0.0     0.0   

Province_State                                       ...       Wyoming         \
Admin2           Butler   Calhoun Chambers Cherokee  ...          Park Platte   
2023-01-30      0.00012  0.000047      0.0      0.0  ...  4.000000e-07    0.0   
2023-01-31      0.00006  0.000023      0.0      0.0  ...  2.000000e-07    0.0   
2023-02-01      0.00003  0.000012      0.0      0.0  ...  1.000000e-07    0.0   

Province_State                                                          \
Admin2          Sheridan Sublette    Sweetwater Teton Uinta Unassigned   
2023-01-30      0.000000      0.0  4.000000e-07   0.0   0.0       -1.0   
2023-01-31      0.005747      0.0  2.000000e-07   0.0   0.0       -1.0   
2023-02-01      0.002874      0.0  1.000000e-07   0.0   0.0       -1.0   

Province_State                  
Admin2         Washakie Weston  
2023-01-30          0.0    0.0  
2023-01-31          0.0    0.0  
2023-02-01          0.0    0.0  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 